In [1]:
from csv import DictReader

with open('fp_cu_products.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows = [r for r in reader]
csv_file.close()

smiles = [r['SMILES'] for r in rows]

In [2]:
from ecnet.datasets import QSPRDataset
from copy import deepcopy

ds = QSPRDataset(smiles, [0.0 for _ in range(len(smiles))], backend='alvadesc')

with open('models/cn/desc.txt', 'r') as f:
    desc_cn = [int(i) for i in f.readlines()]
f.close()
with open('models/ysi/desc.txt', 'r') as f:
    desc_ysi = [int(i) for i in f.readlines()]
f.close()
with open('models/kv/desc.txt', 'r') as f:
    desc_kv = [int(i) for i in f.readlines()]
f.close()
with open('models/cp/desc.txt', 'r') as f:
    desc_cp = [int(i) for i in f.readlines()]
f.close()
with open('models/lhv/desc.txt', 'r') as f:
    desc_lhv = [int(i) for i in f.readlines()]
f.close()
with open('models/fp/desc.txt', 'r') as f:
    desc_fp = [int(i) for i in f.readlines()]
f.close()

ds_cn = deepcopy(ds)
ds_cn.set_desc_index(desc_cn)

ds_ysi = deepcopy(ds)
ds_ysi.set_desc_index(desc_ysi)

ds_kv = deepcopy(ds)
ds_kv.set_desc_index(desc_kv)

ds_cp = deepcopy(ds)
ds_cp.set_desc_index(desc_cp)

ds_lhv = deepcopy(ds)
ds_lhv.set_desc_index(desc_lhv)

ds_fp = deepcopy(ds)
ds_fp.set_desc_index(desc_fp)

In [3]:
from ecnet.model import load_model
import numpy as np

pred_cn = []
pred_ysi = []
pred_kv = []
pred_cp = []
pred_lhv = []
pred_fp = []

for i in range(25):

    model_cn = load_model(f'models/cn/cn_{i}.pt')
    model_ysi = load_model(f'models/ysi/ysi_{i}.pt')
    model_kv = load_model(f'models/kv/kv_{i}.pt')
    model_cp = load_model(f'models/cp/cp_{i}.pt')
    model_lhv = load_model(f'models/lhv/lhv_{i}.pt')
    model_fp = load_model(f'models/fp/fp_{i}.pt')

    pred_cn.append(model_cn(ds_cn.desc_vals).detach().numpy())
    pred_ysi.append(model_ysi(ds_ysi.desc_vals).detach().numpy())
    pred_kv.append(model_kv(ds_kv.desc_vals).detach().numpy())
    pred_cp.append(model_cp(ds_cp.desc_vals).detach().numpy())
    pred_lhv.append(model_lhv(ds_lhv.desc_vals).detach().numpy())
    pred_fp.append(model_fp(ds_fp.desc_vals).detach().numpy())

pred_cn = np.asarray(pred_cn)
pred_ysi = np.asarray(pred_ysi)
pred_kv = np.asarray(pred_kv)
pred_cp = np.asarray(pred_cp)
pred_lhv = np.asarray(pred_lhv)
pred_fp = np.asarray(pred_fp)

In [4]:
mean_cn = np.mean(pred_cn, axis=0)
std_cn = np.std(pred_cn, axis=0)

mean_ysi = np.mean(pred_ysi, axis=0)
std_ysi = np.std(pred_ysi, axis=0)

mean_kv = np.mean(pred_kv, axis=0)
std_kv = np.std(pred_kv, axis=0)

mean_cp = np.mean(pred_cp, axis=0)
std_cp = np.std(pred_cp, axis=0)

mean_lhv = np.mean(pred_lhv, axis=0)
std_lhv = np.std(pred_lhv, axis=0)

mean_fp = np.mean(pred_fp, axis=0)
std_fp = np.std(pred_fp, axis=0)

In [5]:
from csv import DictWriter

for i, row in enumerate(rows):
    row['Pred. CN'] = mean_cn[i][0]
    row['Stdev. CN'] = std_cn[i][0]
    row['Pred. YSI'] = mean_ysi[i][0]
    row['Stdev. YSI'] = std_ysi[i][0]
    row['Pred. KV'] = mean_kv[i][0]
    row['Stdev. KV'] = std_kv[i][0]
    row['Pred. CP'] = mean_cp[i][0]
    row['Stdev. CP'] = std_cp[i][0]
    row['Pred. LHV'] = mean_lhv[i][0]
    row['Stdev. LHV'] = std_lhv[i][0]
    row['Pred. FP'] = mean_fp[i][0]
    row['Stdev. FP'] = std_fp[i][0]

headers = list(rows[0].keys())

with open('fp_cu_predictions.csv', 'w', encoding='utf8') as csv_file:
    writer = DictWriter(csv_file, headers, delimiter=',', lineterminator='\n')
    writer.writeheader()
    writer.writerows(rows)
csv_file.close()